In [1]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
torch.set_printoptions(precision=4, sci_mode=False)

 # <font color = 'dodgerblue'>**Create Tensor**
 Create a torch Tensor of shape (5, 3) which is filled with zeros. Modify the tensor to set element (0, 2) to 10 and element (2, 0)  to 100.

In [3]:
# code here
ten = torch.zeros(5, 3)
print(ten)
ten[0,2] = 10
ten[2, 0] = 100
ten

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])

# <font color = 'dodgerblue'> **Reshape tensor**
You have following tensor as input:

```x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])```

Using only reshaping functions (like view, reshape, transpose, permute), you need to get at the following tensor as output:

```
tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])
```



In [4]:
x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [5]:
# code here
x = torch.reshape(x, (6, 4))
x = torch.transpose(x, 0, 1)
x

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])

# <font color = 'dodgerblue'>**Speedtest for vectorization** </font>

Your goal is to measure the speed of linear algebra operations for different levels of vectorization.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $1024 \times 1024$.
1. Compute $C = A B$ using matrix-matrix operations and report the time. (Hint: Use torch.mm)
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time. (hint use torch.mv inside a for loop)
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time. (Hint: use torch.dot inside nested for loop)

In [6]:
## Solution 1
torch.manual_seed(42) # dod not chnage this
A = torch.empty(1024, 1024)
B = torch.empty(1024, 1024)

In [7]:
## Solution 2
start=time.time()

C = torch.mm(A, B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.14882206916809082 seconds


In [8]:
## Solution 3
C= torch.empty(1024,1024)
start = time.time()

# b = torch.flatten(B)
# print(b.shape)
for i, x in enumerate(B):
  C[:, i] = torch.mv(A, B[:, i])

#print(C)

print("Matrix by vector: " + str(time.time()-start) + " seconds")

Matrix by vector: 0.2884559631347656 seconds


In [9]:
## Solution 4
C= torch.empty(1024,1024)
start = time.time()

for i, x in enumerate(A):
    for j, y in enumerate(B):
        C[i, j] = torch.dot(A[i], B[j])

print("vector by vector: " + str(time.time()-start) + " seconds")

vector by vector: 23.201615571975708 seconds


# <font color = 'dodgerblue'>**Redoing above example using GPU**

<font size = 4, color = 'dodgerblue'> **Using GPUs**

How to use GPUs in Google Colab<br>
In Google Colab -- Go to Runtime Tab at top -- select change runtime type -- for hardware accelartor choose GPU

In [10]:
# Check if GPU is availaible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [11]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((1024, 1024),device=device)
B= torch.randn((1024, 1024),device=device)

In [12]:
## Solution 2
start=time.time()

C = torch.mm(A, B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.05238819122314453 seconds


In [13]:
## Solution 3
C= torch.empty(1024,1024, device = device)
start = time.time()

for i, x in enumerate(B):
  C[:, i] = torch.mv(A, B[:, i])

print("Matrix by vector: " + str(time.time()-start) + " seconds")

Matrix by vector: 0.38759446144104004 seconds


In [14]:
## Solution 4
C= torch.empty(1024,1024, device = device)
start = time.time()

for i, x in enumerate(A):
    for j, y in enumerate(B):
        C[i, j] = torch.dot(A[i], B[j])

print("vector by vector: " + str(time.time()-start) + " seconds")

vector by vector: 18.76826572418213 seconds


# <font color = 'dodgerblue'>**Memory efficient computation**</font>

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible. Hint: If you implement this correcly the memory location of C given by id(C) will be same in both the cells below.

In [15]:
A= torch.randn((1000, 1000), device=device)
B= torch.randn((1000,1000), device=device)
C= torch.randn((1000, 1000), device=device)
print(id(C))

133451430532800


In [16]:
C.addmm_(A, B)

print(id(C))

133451430532800


<font color = 'dodgerblue'>**Broadcast Operations**

In order to perform polynomial fitting we want to compute a design matrix $A$ with

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 3$ and $x = \{1,2,3,4,5\}$. Implement code that generates following A matrix

$$\begin{bmatrix} 1 & 1 & 1 \\ 2 & 4 & 8 \\ 3 & 9 & 27 \\ 4 & 16 & 64 \\ 5 & 25 & 125\end{bmatrix}$$

In [17]:
broad = torch.arange(1., 6.)
#a = torch.square(broad)
exp = torch.tensor([2, 2, 2, 2, 2])
cub = torch.tensor([3, 3, 3, 3, 3])
sq = torch.pow(broad, exp)
cube = torch.pow(broad, cub)
broad = torch.stack((broad, sq, cube), dim = 0)
broad = torch.transpose(broad, 0, 1)
broad

tensor([[  1.,   1.,   1.],
        [  2.,   4.,   8.],
        [  3.,   9.,  27.],
        [  4.,  16.,  64.],
        [  5.,  25., 125.]])

# <font color = 'dodgerblue'> **Image Classification using - Pixel Similarity**

## <font color = 'dodgerblue'> **Import libraries**

In [18]:
# import libraries
import torchvision
import torchvision.transforms as transforms
from pathlib import Path
import cv2 as cv
from google.colab.patches import cv2_imshow # for image display
import pandas as pd

## <font color = 'dodgerblue'> **Mount Google Deive**

In [19]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
data_folder = Path('/content/drive/MyDrive/Colab Notebooks/BUAN 6382/Data')

## <font color = 'dodgerblue'>**Download MNIST training Data**
The MNIST is a dataset of  handwritten digits, available from this page http://yann.lecun.com/exdb/mnist/. It has a training set of 60,000 examples, and a test set of 10,000 examples. In this task we will use images for digits 3 and 7.

In [21]:
trainset = torchvision.datasets.MNIST(root = data_folder,
                                      train = True,
                                      download = True,
                                      )

## <font color = 'dodgerblue'>**Subset of images**

trainset.data has images and trainset.targets has the labels. Below we will create a Boolean mask for digits 3 and 7. We will use this mask to extract subset of images.

In [22]:
# get the mask
idx3 = trainset.targets==3
idx7 = trainset.targets==7

In [23]:
idx3 # This take the value of True wherever the label was 3 and False otherwise

tensor([False, False, False,  ..., False, False, False])

In [24]:
threes = trainset.data[idx3] # use Boolean mask to extract images for digit three
sevens = trainset.data[idx7] # use Boolean mask to extract images for digit seven

In [25]:
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [26]:
type(threes[0])

torch.Tensor

In [27]:
# get a sample image
sample_img_3 = threes[0]

In [28]:
sample_img_3.shape

torch.Size([28, 28])

In [29]:
# get a sample image
sample_img_7 = sevens[0]
sample_img_7.shape

torch.Size([28, 28])

### <font color = 'dodgerblue'>**Task1**
Reshape the image so that it has following  shape: 28, 28, 1. Then convert the tensor to numpy array.

In [30]:
sample_img_3_numpy =  sample_img_3.unsqueeze(2).numpy()
#sample_img_3_numpy

In [31]:
# let us look at the image
# the above steps were needed because the function cv2_imshow needs a three dimenional numpy array of the
# shape H x W  X C (height, width, number of channels). The number of channels for black and white images is 1.
cv2_imshow(sample_img_3_numpy)

### <font color = 'dodgerblue'>**Task2**

Repeat the above steps to show the first image for digit 7.
 - Get the first image.
 - Reshape the image to 28, 28, 1
 - Convert the tensor to numpy array


In [32]:
sample_img_7_numpy =  sample_img_7.unsqueeze(2).numpy()
cv2_imshow(sample_img_7_numpy)

## <font color = 'dodgerblue'>**Visualizing images as pixel values**

<font color = 'dodgerblue'>
- In a computer, images are stored in the form of matrices. The numbers in the matrix are called "pixel values."
- These pixel values show how bright each pixel is.
- 0 means "black," and 255 means "white."
- The matrix of numbers is called the channel, and there is only one channel in a grayscale image.
- For color inages , we have three channels(RGB - Red, Green Blue). That is we need three matrices to store color images.
</font>

In [33]:
df = pd.DataFrame(sample_img_3[5:24, 7:22].numpy())
df.style.set_properties(**{'font-size':'8pt'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


<font color = 'dodgerblue'> - In the matrix below we have flipped color just for illustartion (We are using 0 for white instead of black).

In [34]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


### <font color = 'dodgerblue'>**Step 3**

<font color = 'dodgerblue'>**Convert threes and sevens generated earlier to float and and take the mean along dimension 0.**

Here we are generating an average image by taking the average of each pixel across images.

In [35]:
mean_threes =  threes.mean(dim=0, dtype=torch.float32)
mean_sevens =  sevens.mean(dim=0, dtype=torch.float32)

In [36]:
mean_threes.shape
mean_sevens.shape

torch.Size([28, 28])

In [37]:
cv2_imshow(mean_threes.unsqueeze(dim=2).numpy())
print()
cv2_imshow(mean_sevens.unsqueeze(dim=2).numpy())

## <font color = 'dodgerblue'>**Prediction for validation images**

We will calculate the distance of each validation image from the mean_threes and mean_sevens. If the validation image is closest to mean_threes then we will predict 1 else we will predict 0.

## <font color = 'dodgerblue'>**Get Valid Dataset**

In [38]:
validset = torchvision.datasets.MNIST(root = data_folder,
                                      train = False,
                                      download = True,
                                      )

In [39]:
# get the mask
idx3 = validset.targets==3
idx7 = validset.targets==7

In [40]:
validset.data.shape

torch.Size([10000, 28, 28])

In [41]:
# get images and labels using the mask
valid_data_3_7 = validset.data[idx3+idx7]
valid_targets_3_7 = validset.targets[idx3 + idx7]

In [42]:
# check the shape of the inputs (images)
# we have 2038 images with size 28 x 28.
valid_data_3_7.shape

torch.Size([2038, 28, 28])

In [43]:
# lables for valid dataset
valid_targets_3_7

tensor([7, 7, 3,  ..., 3, 7, 3])

In [44]:
# change the lable to 1 where the label was 3
valid_targets_3_7[valid_targets_3_7==3] = 1

In [45]:
# change the lable to 0 where the label was 7
valid_targets_3_7[valid_targets_3_7==7] = 0

In [46]:
valid_targets_3_7.unique()

tensor([0, 1])

In [47]:
valid_targets_3_7

tensor([0, 0, 1,  ..., 1, 0, 1])

In [48]:
valid_data_3_7.shape

torch.Size([2038, 28, 28])

## <font color = 'dodgerblue'>**Distance between images**

### <font color = 'dodgerblue'>**Task4**

<font color = 'dodgerblue'>**Write a function to calculate the distance between two images a and b.**
 - Calculate the difference between corresponding pixels of two images.
 - Calculate square of differences.
 - Take the mean (take the mean across last two dimensions (-1, -2) of the square of the differences and then take the square root.

In [49]:
def dist_images(a, b):

  subt = a.float() - b.float()
  sq = subt ** 2
  mean_dist = torch.sqrt(sq.mean((-1, -2)))
  return mean_dist

In [50]:
# calculate the distance of the each image in validation set from mean_threes
# bacause of the  broadcasting we are able to use the above function which was written to
# calculate the distnace between two images
valid_3_dist = dist_images(valid_data_3_7, mean_threes)

In [51]:
# calculate the distance of the each image in validation set from mean_sevens
valid_7_dist = dist_images(valid_data_3_7, mean_sevens)

In [52]:
# if you have done everything right, you will observe following shapes.
valid_3_dist.shape, valid_7_dist.shape

(torch.Size([2038]), torch.Size([2038]))

## <font color = 'dodgerblue'>**Prediction Using Distance**

### <font color = 'dodgerblue'>**Task5**



In [53]:
# step 1 : generate the tensor filled with zeros that has the same shape as valid_target_3_7
prediction =  torch.zeros_like(valid_targets_3_7)

In [54]:
# step 2: update the tensor prediction. Update the value to 1, when valid_3_dist < valid_7_dist.
# Hint (Generate a Boolean mask using the condition valid_3_dist < valid_7_dist and use the mask to update value to 1 )

cond = (valid_3_dist < valid_7_dist)
prediction[cond] = 1

In [55]:
prediction

tensor([0, 0, 1,  ..., 1, 0, 1])

## <font color = 'dodgerblue'>**Accuracy**

In [56]:
correct = prediction == valid_targets_3_7

In [57]:
correct

tensor([True, True, True,  ..., True, True, True])

In [58]:
100 * correct.float().mean().item()

96.6143250465393